In [10]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def Weight_Variable(shape):
    tfV_initial = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    return tfV_initial

# 生成偏差
def Bias_Vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

#卷积层
def Conv2d(x, W):
    #ksize必须是[1, x, y, 1]
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#池化层
def Max_pool_2x2(x):
    #ksize [1, x, y, 1]
    max_pool_ret = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],\
            strides=[1, 2, 2, 1], padding = 'SAME')
    return max_pool_ret

# 记录变量
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [12]:
#region #这里开始主程序 __name__ == "__main__"
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

#改变x的格式，转为4D的向量
# -1:不限制图片数量，28,28:28x28尺寸，1:灰度图，只有1个Channel
x_image = tf.reshape(x, [-1, 28, 28, 1])

#region #初始化第1层卷积层的连接权矩阵和偏置
W_conv1 = Weight_Variable([5, 5, 1, 32])  # 5*5的卷积核, 32个核从1个平面抽取特征
b_conv1 = Bias_Vairable([32])  # 共享参数，每个卷积核都用1个偏置，32个卷积核，就32个偏置
#把2维的x_image进行卷积
h_conv1 = tf.nn.relu(Conv2d(x_image, W_conv1) + b_conv1)
#进行pooling
h_pool1 = Max_pool_2x2(h_conv1)
#endregion

#region #初始化第2层卷积层的连接权矩阵和偏置
W_conv2 = Weight_Variable([5, 5, 32, 64])  # 5*5的卷积核, 64个核从32个平面抽取特征
b_conv2 = Bias_Vairable([64])  # 共享参数，每个卷积核都用1个偏置，64个卷积核，就64个偏置
#把2维的x_image进行卷积
h_conv2 = tf.nn.relu(Conv2d(h_pool1, W_conv2) + b_conv2)  # 变成14*14
#进行pooling
h_pool2 = Max_pool_2x2(h_conv2)  # 变成7*7
#endregion

#region #定义第1层全连接层FC1
# 上一层有7*7的64个图，因此展开成1维长度就是7*7*64
# 然后FC1层有1024个神经元，所以连接权值的维度是[7*7*64, 1024]
W_fc1 = Weight_Variable([7*7*64, 128])
b_fc1 = Bias_Vairable([128])
#进行FC1的运算定义
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  # tf.Variable可以直接相加
#引入神经元的输出率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
#endregion

#region #进行FC2的运算定义。FC2有10个神经元，采用softmax，one-hot形式，对应10个分类
W_fc2 = Weight_Variable([128, 10])
b_fc2 = Bias_Vairable([10])
#计算输出
out = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
#endregion


In [13]:
#交叉熵代价函数
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=out))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#
correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
#
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
log_dir = "Logs/log-6.1"
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size
# 有85%的神经元激活
my_keep_prob = 0.85
Max_Train_Count = 21 #整个数据集反复训练的次数
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(Max_Train_Count):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)            
            sess.run(train_step,
                feed_dict={x: batch_xs, y: batch_ys, keep_prob: my_keep_prob})
        if(epoch % 2 == 0):
            acc = sess.run(accuracy,
                feed_dict={x: batch_xs, y: batch_ys, keep_prob: my_keep_prob})
            print("training round=", epoch, " and accuracy=", acc)
    
    #最后用测试集数据来验证模型的准确率. 这一步因为数据量大，容易耗费显存。
    acc = sess.run(accuracy,
            feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    print('Test the model, Acc=', acc)
print('Finished')

training round= 0  and accuracy= 0.79
training round= 2  and accuracy= 0.97
training round= 4  and accuracy= 0.97
training round= 6  and accuracy= 0.97
training round= 8  and accuracy= 0.98
training round= 10  and accuracy= 1.0
training round= 12  and accuracy= 0.98
training round= 14  and accuracy= 0.99
training round= 16  and accuracy= 0.99
training round= 18  and accuracy= 0.99
training round= 20  and accuracy= 0.98
Test the model, Acc= 0.9896
Finished
